## L'artificialisation du littoral d'après la BD Topo

*Denis Vannier - Le Plan Studio - 2024 - https://leplan.studio/ - https://github.com/Denis-Vannier*

Source : IGN, BD Topo : https://geoservices.ign.fr/bdtopo

In [54]:
import pandas as pd
pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)
pd.set_option('display.float_format', lambda x: '%.9f' % x)
import requests
import datetime
import matplotlib.pyplot as plt
import geopandas as gpd
import mapclassify
import os
import numpy as np
import shapely
from shapely.geometry import Polygon, Point
from shapely.validation import make_valid


In [56]:
# Chargement des polygones des bandes littorales :
Bande10kmComLitt=gpd.read_file("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp")
Bande10kmComLitt=Bande10kmComLitt.to_crs("EPSG:2154")
Bande10kmComLitt["INSEE_COM"]=Bande10kmComLitt["INSEE_COM"].astype("str")
ComLitt=Bande10kmComLitt["INSEE_COM"].tolist()
Bande10kmComLitt_dissolve=Bande10kmComLitt[["geometry"]]
Bande10kmComLitt_dissolve=Bande10kmComLitt_dissolve.dissolve()
Bande10kmComLitt.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1994 entries, 0 to 1993
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   BANDE_LITT  1994 non-null   object  
 1   INSEE_COM   1994 non-null   object  
 2   NOM         1994 non-null   object  
 3   SIREN_EPCI  1994 non-null   object  
 4   NOM_EPCI    1984 non-null   object  
 5   numdep      1994 non-null   object  
 6   SURFACE_M2  1994 non-null   float64 
 7   geometry    1994 non-null   geometry
dtypes: float64(1), geometry(1), object(6)
memory usage: 124.8+ KB


### Création de la couche artificialisée à partir des bâtiments et des voies de communication

### 2014

In [58]:
dep=["56","22","29","44"]

for dep in dep:
    ####################  Batiments 2014 ##########################
    batA2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/BATI_INDIFFERENCIE.SHP")
    batB2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/BATI_INDUSTRIEL.SHP")
    batC2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/CIMETIERE.SHP")
    batD2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/BATI_REMARQUABLE.SHP")
    batE2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/CONSTRUCTION_SURFACIQUE.SHP")
    batF2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/RESERVOIR.SHP")
    batG2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/PISTE_AERODROME.SHP")
    batH2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/CONSTRUCTION_LEGERE.SHP")
    batI2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/E_BATI/TERRAIN_SPORT.SHP")
    bat2014=gpd.GeoDataFrame(pd.concat([batA2014,batB2014,batC2014,batD2014,batE2014,batF2014,batG2014,batH2014,batI2014], ignore_index=True))
    bat2014=gpd.GeoDataFrame(bat2014, crs="EPSG:2154")
    bat2014_com=bat2014.sjoin(Bande10kmComLitt)
    bat2014_com=bat2014_com[['INSEE_COM','NOM','geometry']]
    bat2014_com.to_file("BD_TOPO/Temp/BATIMENTS_BRUTS/BAT_COM_LITT_"+dep+"_2014.shp")

    BatDep2014=bat2014_com[['geometry']]
    BatDep2014['buffered']=BatDep2014.buffer(50.0, join_style=1 )
    BatDep2014BUFF1=BatDep2014.set_geometry("buffered")
    BatDep2014BUFF1=gpd.GeoDataFrame(BatDep2014BUFF1,geometry='buffered', crs='EPSG:2154')
    BatDep2014BUFF1=BatDep2014BUFF1.dissolve()
    BatDep2014BUFFok=BatDep2014BUFF1.buffer(-45, join_style=1 )
    BatDep2014BUFFok=BatDep2014BUFFok.reset_index()
    BatDep2014BUFFok.rename(columns={0:"geometry"}, inplace=True)
    BatDep2014BUFFok=gpd.GeoDataFrame(BatDep2014BUFFok,geometry='geometry', crs='EPSG:2154')
    BatDep2014BUFFok=BatDep2014BUFFok[['geometry']]
    BatDep2014BUFFok=BatDep2014BUFFok.dissolve()
    BatDep2014BUFFok.to_file("BD_TOPO/Temp/BATIMENTS_BUFFER/BAT_BUFFER_"+dep+"_2014.shp")

    BatDep2014BUFFok=gpd.read_file("BD_TOPO/Temp/BATIMENTS_BUFFER/BAT_BUFFER_"+dep+"_2014.shp")

    ####################  Routes 2014 ##########################
    Route2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/A_RESEAU_ROUTIER/ROUTE.SHP")
    # On ne garde pas les sentiers, les chemins et les pistes cyclables :
    Route2014=Route2014[Route2014["NATURE"]!="Sentier"]
    Route2014=Route2014[Route2014["NATURE"]!="Chemin"]
    Route2014=Route2014[Route2014["NATURE"]!="Piste cyclable"]
    # Application d'un buffer selon le type de route :
    # L'emprise des infrastructures routières provient des estimations publiées en 2013 
    # par le « groupe de travail national sur l’évaluation de la consommation d’espace à partir des fichiers fonciers »
    # http://piece-jointe-carto.developpement-durable.gouv.fr/REG074B/FONCIER_SOL/N_OCCUPATION_SOL/L_EMPRISE_ROUTE_R74/Fiche1-7-1.pdf
    # L'emprise corrigée des ponts (variable "FRANCHISST") n'a pas été prise en compte, pour permettre une comparaisons avec les millésimes plus récents de la BD Topo.
    Route2014.loc[(Route2014["NATURE"]=="Quasi-autoroute") & (Route2014["NB_VOIES"]==2), "buff"] = 5
    Route2014.loc[(Route2014["NATURE"]=="Quasi-autoroute") & (Route2014["NB_VOIES"]==3), "buff"] = 7
    Route2014.loc[(Route2014["NATURE"]=="Quasi-autoroute") & (Route2014["NB_VOIES"]==4), "buff"] = 8
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==0), "buff"] = 4.5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==1), "buff"] = 4.5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==2), "buff"] = 6
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Escalier") & (Route2014["NB_VOIES"]==0), "buff"] = 1
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==0), "buff"] = 3.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==1), "buff"] = 3.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==2), "buff"] = 3.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==3), "buff"] = 3.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==4), "buff"] = 3.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==5), "buff"] = 3.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==0), "buff"] = 4
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==1), "buff"] = 4
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==2), "buff"] = 4.5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Autre") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==3), "buff"] = 4.5
    Route2014.loc[Route2014["NATURE"]=="Route empierrée", "buff"] = 1
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==0), "buff"] = 3.75
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==1), "buff"] = 3.75
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==2), "buff"] = 5.5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==0), "buff"] = 3
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==1), "buff"] = 3
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==2), "buff"] = 5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==3), "buff"] = 5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==4), "buff"] = 5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==0), "buff"] = 4.5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==1), "buff"] = 4.5
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==2), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Départementale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==3), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==1), "buff"] = 5.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Bretelle") & (Route2014["NB_VOIES"]==2), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==1), "buff"] = 5.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==2), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==3), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 1 chaussée") & (Route2014["NB_VOIES"]==4), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==0), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==1), "buff"] = 5.25
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==2), "buff"] = 7
    Route2014.loc[(Route2014["CL_ADMIN"]=="Nationale") & (Route2014["NATURE"]=="Route à 2 chaussées") & (Route2014["NB_VOIES"]==3), "buff"] = 7
    Route2014["buff"].fillna("1", inplace=True)
    Route2014["buff"]=Route2014["buff"].astype("float64")
    Route2014["buffered"]=Route2014.buffer(Route2014["buff"], join_style=1)
    Route2014=Route2014[["buffered"]]
    Route2014BUFF=gpd.GeoDataFrame(Route2014,geometry='buffered', crs='EPSG:2154')
    Route2014BUFF=Route2014BUFF.dissolve()
    Route2014BUFF.rename(columns={"buffered":"geometry"}, inplace=True)
    Route2014BUFF=Route2014BUFF.set_geometry("geometry")

    ########################## Voies ferrées 2014 ##########################
    VoieFerree2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/B_VOIES_FERREES_ET_AUTRES/TRONCON_VOIE_FERREE.SHP")
    # Lister les valeurs de buffer à appliquer en fonction du nombre de voies : 
    VoieFerree2014.loc[VoieFerree2014["NB_VOIES"]==1, "buff"] = 4.2
    VoieFerree2014.loc[VoieFerree2014["NB_VOIES"]==2, "buff"] = 5.9
    VoieFerree2014.loc[VoieFerree2014["NB_VOIES"]==3, "buff"] = 7.6
    VoieFerree2014.loc[VoieFerree2014["NB_VOIES"]==4, "buff"] = 9.3
    VoieFerree2014["buff"].fillna("1", inplace=True)
    VoieFerree2014["buff"]=VoieFerree2014["buff"].astype("float64")
    VoieFerree2014["buffered"]=VoieFerree2014.buffer(VoieFerree2014["buff"], join_style=1 )
    VoieFerree2014=VoieFerree2014[["buffered"]]
    
    VoieFerree2014BUFF=gpd.GeoDataFrame(VoieFerree2014,geometry="buffered", crs="EPSG:2154")
    VoieFerree2014BUFF=VoieFerree2014BUFF.dissolve()
    VoieFerree2014BUFF.rename(columns={"buffered":"geometry"}, inplace=True)
    VoieFerree2014BUFF=VoieFerree2014BUFF.set_geometry("geometry")

    ########################## Parkings, gares, aires de triage 2014 ##########################
    Parking2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/A_RESEAU_ROUTIER/SURFACE_ROUTE.SHP")
    Gare2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/B_VOIES_FERREES_ET_AUTRES/GARE.SHP")
    Triage2014=gpd.read_file("BD_TOPO/BDTOPO_2014/BDTOPO_2-1_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2014-04-02/BDTOPO/1_DONNEES_LIVRAISON_2021-09-00173/BDT_2-1_SHP_LAMB93_D0"+dep+"-ED141/B_VOIES_FERREES_ET_AUTRES/AIRE_TRIAGE.SHP")
    Parking2014=pd.concat([Parking2014, Gare2014, Triage2014])
    Parking2014=Parking2014[["geometry"]]

    Artif2014=pd.concat([BatDep2014BUFFok, Route2014BUFF, VoieFerree2014BUFF, Parking2014])
    Artif2014_ok=gpd.GeoDataFrame(geometry=[Artif2014.unary_union], crs="EPSG:2154").explode(index_parts=False).reset_index(drop=True)
    Artif2014_ok=Artif2014_ok[["geometry"]]
    Artif2014_ok=Artif2014_ok.dissolve()
    Artif2014_ok=Artif2014_ok.overlay(Bande10kmComLitt_dissolve, how='intersection', keep_geom_type=None, make_valid=True)
    Artif2014_ok.to_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_"+dep+"_2014_OK.shp")


/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37896/4281919854.py:88: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace o

### 2024

In [64]:
dep=["56","22","29","35","44"]

for dep in dep:

    ####################  Batiments 2024 ##########################
    batA2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/BATI/BATIMENT.shp")
    batB2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/BATI/CIMETIERE.shp")
    batC2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/BATI/CONSTRUCTION_SURFACIQUE.shp")
    batD2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/BATI/RESERVOIR.shp")
    bat2024=gpd.GeoDataFrame(pd.concat([batA2024,batB2024,batC2024,batD2024], ignore_index=True))
    bat2024=gpd.GeoDataFrame(bat2024, crs="EPSG:2154")
    bat2024_com=bat2024.sjoin(Bande10kmComLitt)
    bat2024_com=bat2024_com[['INSEE_COM','NOM','geometry']]
    bat2024_com.to_file("BD_TOPO/Temp/BATIMENTS_BRUTS/BAT_COM_LITT_"+dep+"_2024.shp")

    BatDep2024=bat2024_com[['geometry']]
    BatDep2024['buffered']=BatDep2024.buffer(50.0, join_style=1 )
    BatDep2024BUFF1=BatDep2024.set_geometry("buffered")
    BatDep2024BUFF1=gpd.GeoDataFrame(BatDep2024BUFF1,geometry='buffered', crs='EPSG:2154')
    BatDep2024BUFF1=BatDep2024BUFF1.dissolve()
    BatDep2024BUFFok=BatDep2024BUFF1.buffer(-45, join_style=1 )
    BatDep2024BUFFok=BatDep2024BUFFok.reset_index()
    BatDep2024BUFFok.rename(columns={0:"geometry"}, inplace=True)
    BatDep2024BUFFok=gpd.GeoDataFrame(BatDep2024BUFFok,geometry='geometry', crs='EPSG:2154')
    BatDep2024BUFFok=BatDep2024BUFFok[['geometry']]
    BatDep2024BUFFok=BatDep2024BUFFok.dissolve()
    BatDep2024BUFFok.to_file("BD_TOPO/Temp/BATIMENTS_BUFFER/BAT_BUFFER_"+dep+"_2024.shp")

    BatDep2024BUFFok=gpd.read_file("BD_TOPO/Temp/BATIMENTS_BUFFER/BAT_BUFFER_"+dep+"_2024.shp")
    
    # ####################  Routes 2024 ##########################
    Route2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/TRANSPORT/TRONCON_DE_ROUTE.shp")
    # On ne garde pas les sentiers, les chemins et les pistes cyclables :
    Route2024=Route2024[Route2024["NATURE"]!="Sentier"]
    Route2024=Route2024[Route2024["NATURE"]!="Chemin"]
    Route2024=Route2024[Route2024["NATURE"]!="Piste cyclable"]
    # On applique un buffer en fonction du type de routes :
    # L'emprise des infrastructures routières provient des estimations publiées en 2013 
    # par le « groupe de travail national sur l’évaluation de la consommation d’espace à partir des fichiers fonciers » : 
    # http://piece-jointe-carto.developpement-durable.gouv.fr/REG074B/FONCIER_SOL/N_OCCUPATION_SOL/L_EMPRISE_ROUTE_R74/Fiche1-7-1.pdf
    # L'emprise corrigée des ponts (variable "FRANCHISST") n'a pas été prise en compte, pour permettre une comparaisons avec les millésimes plus récents de la BD Topo.
    Route2024.loc[(Route2024["NATURE"]=="Type autoroutier") & (Route2024["NB_VOIES"]==2), "buff"] = 5
    Route2024.loc[(Route2024["NATURE"]=="Type autoroutier") & (Route2024["NB_VOIES"]==3), "buff"] = 7
    Route2024.loc[(Route2024["NATURE"]=="Type autoroutier") & (Route2024["NB_VOIES"]==4), "buff"] = 8
    Route2024.loc[(Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==0), "buff"] = 4.5
    Route2024.loc[(Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==1), "buff"] = 4.5
    Route2024.loc[(Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==2), "buff"] = 6
    Route2024.loc[(Route2024["NATURE"]=="Escalier") & (Route2024["NB_VOIES"]==0), "buff"] = 1
    Route2024.loc[(Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==0), "buff"] = 3.25
    Route2024.loc[(Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==1), "buff"] = 3.25
    Route2024.loc[(Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==2), "buff"] = 3.25
    Route2024.loc[(Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==3), "buff"] = 3.25
    Route2024.loc[(Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==4), "buff"] = 3.25
    Route2024.loc[(Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==5), "buff"] = 3.25
    Route2024.loc[(Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==0), "buff"] = 4
    Route2024.loc[(Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==1), "buff"] = 4
    Route2024.loc[(Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==2), "buff"] = 4.5
    Route2024.loc[(Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==3), "buff"] = 4.5
    Route2024.loc[Route2024["NATURE"]=="Route empierrée", "buff"] = 1
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==0), "buff"] = 3.75
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==1), "buff"] = 3.75
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==2), "buff"] = 5.5
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==0), "buff"] = 3
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==1), "buff"] = 3
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==2), "buff"] = 5
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==3), "buff"] = 5
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==4), "buff"] = 5
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==0), "buff"] = 4.5
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==1), "buff"] = 4.5
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==2), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Départementale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==3), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==1), "buff"] = 5.25
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Bretelle") & (Route2024["NB_VOIES"]==2), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==1), "buff"] = 5.25
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==2), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==3), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 1 chaussée") & (Route2024["NB_VOIES"]==4), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==0), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==1), "buff"] = 5.25
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==2), "buff"] = 7
    Route2024.loc[(Route2024["CL_ADMIN"]=="Nationale") & (Route2024["NATURE"]=="Route à 2 chaussées") & (Route2024["NB_VOIES"]==3), "buff"] = 7
    Route2024["buff"].fillna("1", inplace=True)
    Route2024["buff"]=Route2024["buff"].astype("float64")
    Route2024["buffered"]=Route2024.buffer(Route2024["buff"], join_style=1)
    Route2024=Route2024[["buffered"]]
    Route2024BUFF=gpd.GeoDataFrame(Route2024,geometry='buffered', crs='EPSG:2154')
    Route2024BUFF=Route2024BUFF.dissolve()
    Route2024BUFF.rename(columns={"buffered":"geometry"}, inplace=True)
    Route2024BUFF=Route2024BUFF.set_geometry("geometry")


    # ########################## Voies ferrées 2024 ##########################
    VoieFerree2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/TRANSPORT/TRONCON_DE_VOIE_FERREE.shp")
    # Lister les valeurs de buffer à appliquer en fonction du nombre de voies : 
    VoieFerree2024.loc[VoieFerree2024["NB_VOIES"]==1, "buff"] = 4.2
    VoieFerree2024.loc[VoieFerree2024["NB_VOIES"]==2, "buff"] = 5.9
    VoieFerree2024.loc[VoieFerree2024["NB_VOIES"]==3, "buff"] = 7.6
    VoieFerree2024.loc[VoieFerree2024["NB_VOIES"]==4, "buff"] = 9.3
    VoieFerree2024["buff"].fillna("1", inplace=True)
    VoieFerree2024["buff"]=VoieFerree2024["buff"].astype("float64")
    VoieFerree2024["buffered"]=VoieFerree2024.buffer(VoieFerree2024["buff"], join_style=1 )
    VoieFerree2024=VoieFerree2024[["buffered"]]
    
    VoieFerree2024BUFF=gpd.GeoDataFrame(VoieFerree2024,geometry="buffered", crs="EPSG:2154")
    VoieFerree2024BUFF=VoieFerree2024BUFF.dissolve()
    VoieFerree2024BUFF.rename(columns={"buffered":"geometry"}, inplace=True)
    VoieFerree2024BUFF=VoieFerree2024BUFF.set_geometry("geometry")

    ########################## Parkings 2024 ##########################
    Parking2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/TRANSPORT/EQUIPEMENT_DE_TRANSPORT.shp")
    Parking2024=Parking2024[Parking2024["NATURE"]=="Parking"]
    Parking2024=Parking2024[["geometry"]]

    # ########################## Pistes d'aérodromes 2024 ##########################

    Aerodrome2024=gpd.read_file("BD_TOPO/BDTOPO_JUIN_2024/BDTOPO_3-4_TOUSTHEMES_SHP_LAMB93_D0"+dep+"_2024-06-15/BDTOPO/1_DONNEES_LIVRAISON_2024-06-00178/BDT_3-4_SHP_LAMB93_D0"+dep+"-ED2024-06-15/TRANSPORT/PISTE_D_AERODROME.shp")
    Aerodrome2024=Aerodrome2024[["geometry"]]
    

    Artif2024=pd.concat([BatDep2024BUFFok, Parking2024])
    Artif2024_ok=gpd.GeoDataFrame(geometry=[Artif2024.unary_union], crs="EPSG:2154").explode(index_parts=False).reset_index(drop=True)
    Artif2024_ok=Artif2024_ok.dissolve()
    Artif2024_ok=Artif2024_ok.overlay(Bande10kmComLitt_dissolve, how='intersection', keep_geom_type=None, make_valid=True)
    Artif2024_ok.to_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_"+dep+"_2024_OK.shp")

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37896/14037435.py:122: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  Artif2024_ok=gpd.GeoDataFrame(geometry=[Artif2024.unary_union], crs="EPSG:2154").explode(index_parts=False).reset_index(drop=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37896/14037435.py:122: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  Artif2024_ok=gpd.GeoDataFrame(geometry=[Artif2024.unary_union], crs="EPSG:2154").explode(index_parts=False).reset_index(drop=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37896/14037435.py:122: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  Artif2024_ok=gpd.GeoDataFrame(geometry=[Artif2024.unary_union], crs="EPSG:2154").explode(index_parts=False).reset_index(drop=True)
/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipy

### Jointure spatiale des tâches urbaines littorale 2014 et 2023 avec les découpage par communes et par secteurs littoraux :

In [65]:
Bande_Littorale_Com=gpd.read_file("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM.shp")
dep=["56","22","29","35","44"]

In [ ]:
##### 2014 #####
TacheUrb2014_22=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_22_2014_OK.shp")
TacheUrb2014_29=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_29_2014_OK.shp")
TacheUrb2014_56=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_56_2014_OK.shp")
TacheUrb2014_35=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_35_2014_OK.shp")
TacheUrb2014_44=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_44_2014_OK.shp")
TacheUrb2014=gpd.GeoDataFrame(pd.concat([TacheUrb2014_22,TacheUrb2014_29,TacheUrb2014_56,TacheUrb2014_35,TacheUrb2014_44], ignore_index=True))
TacheUrb2014=gpd.GeoDataFrame(TacheUrb2014, crs="EPSG:2154")
TacheUrb2014=TacheUrb2014[["geometry"]]
TacheUrb2014["geometry"]=TacheUrb2014["geometry"].apply(make_valid)
TacheUrb2014_ok=TacheUrb2014.dissolve()
TacheUrb2014_ok.to_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_Brute_2014.shp")


In [66]:
##### 2024 #####
TacheUrb2024_22=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_22_2024_OK.shp")
TacheUrb2024_29=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_29_2024_OK.shp")
TacheUrb2024_56=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_56_2024_OK.shp")
TacheUrb2024_35=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_35_2024_OK.shp")
TacheUrb2024_44=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BRUTE/TACHE_ARTIF_44_2024_OK.shp")
TacheUrb2024=gpd.GeoDataFrame(pd.concat([TacheUrb2024_22,TacheUrb2024_29,TacheUrb2024_56,TacheUrb2024_35,TacheUrb2024_44], ignore_index=True))
TacheUrb2024=gpd.GeoDataFrame(TacheUrb2024, crs="EPSG:2154")
TacheUrb2024=TacheUrb2024[["geometry"]]
TacheUrb2024["geometry"]=TacheUrb2024["geometry"].apply(make_valid)
TacheUrb2024_ok=TacheUrb2024.dissolve()
TacheUrb2024_ok.to_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_Brute_2024.shp")

### Difference tâches 2014-2024
Overlay avec les données 2014, pour conserver uniquement les polygones des zones artificialisées entre 2014 et 2024.

Contenu du fichier makefile, à placer dans le même dossier que ce notebook : 

	clean_artif:
		mapshaper BD_TOPO/Temp/TACHE_ARTIF_BZH_Brute_2014.shp \
		-clean gap-fill-area=5000m2 \
		-simplify 25% weighting=0.7 keep-shapes \
		-clip BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp \
		-o BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp \
		

		mapshaper BD_TOPO/Temp/TACHE_ARTIF_BZH_Brute_2024.shp \
		-clean gap-fill-area=5000m2 \
		-simplify 25% weighting=0.7 keep-shapes \
		-erase BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp \
		-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024.shp

	clip_artif:
		mapshaper BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp \
		-clip BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024.shp \
		-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.shp

		mapshaper BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp \
		-clip BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp \
		-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.shp


In [67]:
!make clean_artif
!make clip_artif

python(27484) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


mapshaper BD_TOPO/Temp/TACHE_ARTIF_BZH_Brute_2014.shp \
	-clean gap-fill-area=5000m2 \
	-simplify 25% weighting=0.7 keep-shapes \
	-clip BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp \
	-o BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp \
	
[i] Warning: Shapefile Z data will be lost.
[clean] Removed 63,419 / 83,920 slivers using 5000+ sqm variable threshold
[clean] Retained 1 of 1 features
[simplify] Repaired 55 intersections
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shx
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.dbf
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.prj
mapshaper BD_TOPO/Temp/TACHE_ARTIF_BZH_Brute_2024.shp \
	-clean gap-fill-area=5000m2 \
	-simplify 25% weighting=0.7 keep-shapes \
	-erase BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp \
	-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024.shp
[i] Warning: Shapefile Z data will be lost.
[clean] Removed 376 / 2,711 slivers using 5000+ sqm variable threshold
[clea

python(27736) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


mapshaper BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp \
	-clip BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024.shp \
	-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.shp
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.shp
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.shx
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.dbf
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.prj
mapshaper BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH.shp \
	-clip BD_TOPO/Temp/TACHE_ARTIF_BZH_CLEAN_2014.shp \
	-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.shp
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.shp
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.shx
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.dbf
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.prj


In [68]:
Artif_2014_2024=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt.shp")
Artif_2014_2024=Artif_2014_2024.explode(ignore_index=True)
Artif_2014_2024["Artif2024_m2"]=Artif_2014_2024["geometry"].area

# Filtrage des polygones d'au moins 500m2 :
Artif_2014_2024=Artif_2014_2024[Artif_2014_2024["Artif2024_m2"]>=500]
Artif_2014_2024.to_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE.geojson")
Artif_2014_2024.to_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE.shp")

/var/folders/zf/sc5qjn6x54b02sbxg3k9jbkr0000gn/T/ipykernel_37896/1985593679.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  Artif_2014_2024.to_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE.shp")
/Users/denisvannier/opt/anaconda3/envs/GEODATA2/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Artif2024_m2' to 'Artif2024_'
  ogr_write(


In [69]:
# Contenu du fichier makefile : 
# mapshaper BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE.shp -simplify 50% -clean -o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE_simplif.shp
! make simplifier

Artif_2014_2024=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE_simplif.geojson")
Artif_2014_2024["ID_COM_LITT"]=Artif_2014_2024["INSEE_COM"]+"_"+Artif_2014_2024["BANDE_LITT"]
Artif_2014_2024=Artif_2014_2024.dissolve(by="ID_COM_LITT", as_index=False)
Artif_2014_2024=gpd.GeoDataFrame(Artif_2014_2024, geometry="geometry", crs="EPSG:2154")

# Le nom de la commune a été modifié durant les transformations avec Mapshaper. 
# On récupère les noms corrects :
epci=pd.read_csv("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH_STATS.csv")
epci=epci[["BANDE_LITT","INSEE_COM", "NOM", 'SIREN_EPCI', 'NOM_EPCI']]
epci["INSEE_COM"]=epci["INSEE_COM"].astype("str")
epci["ID_COM_LITT"]=epci["INSEE_COM"]+"_"+epci["BANDE_LITT"]
epci=epci[['ID_COM_LITT','NOM','SIREN_EPCI', 'NOM_EPCI']]
Artif_2014_2024=Artif_2014_2024[['ID_COM_LITT', 'geometry', 'BANDE_LITT', 'INSEE_COM', 'numdep', 'SURFACE_M2', 'Artif2024_m2']]
Artif_2014_2024=Artif_2014_2024.merge(epci, on="ID_COM_LITT", how="left")

Artif_2014_2024["Artif2024_m2"]=Artif_2014_2024["geometry"].area
Artif_2014_2024=Artif_2014_2024.to_crs("epsg:4326")
Artif_2014_2024.rename(columns={"NOM":"NOM_COMMUNE"}, inplace=True)
Artif_2014_2024=Artif_2014_2024[['ID_COM_LITT', 'BANDE_LITT', 'numdep', 'INSEE_COM', 'NOM_COMMUNE', 'SIREN_EPCI', 'NOM_EPCI', 'SURFACE_M2','Artif2024_m2', 'geometry']]
Artif_2014_2024["SURFACE_M2"]=Artif_2014_2024["SURFACE_M2"].round(0)
Artif_2014_2024["Artif2024_m2"]=Artif_2014_2024["Artif2024_m2"].round(0)

Artif_2014_2024.to_file("BD_TOPO/TACHE_ARTIF_BZH_2014_2024_BandeLitt_DEF.geojson")
Artif_2014_2024.info()

python(29182) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


mapshaper BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE.geojson \
	-simplify 50% \
	-clean \
	-o BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE_simplif.geojson
[simplify] Repaired 1,505 intersections
[clean] Retained 20,383 of 20,385 features
[o] RFC 7946 warning: non-WGS84 GeoJSON output.
[o] Wrote BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_2024_BandeLitt_FILTRE_simplif.geojson
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1588 entries, 0 to 1587
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID_COM_LITT   1588 non-null   object  
 1   BANDE_LITT    1588 non-null   object  
 2   numdep        1588 non-null   object  
 3   INSEE_COM     1588 non-null   object  
 4   NOM_COMMUNE   1588 non-null   object  
 5   SIREN_EPCI    1588 non-null   object  
 6   NOM_EPCI      1583 non-null   object  
 7   SURFACE_M2    1588 non-null   float64 
 8   Artif2024_m2  1588 non-null   float64 
 9   geom

In [70]:
Artif_2014=gpd.read_file("BD_TOPO/Temp/TACHE_ARTIF_BZH_2014_BandeLitt.shp")
Artif_2014["ID_COM_LITT"]=Artif_2014["INSEE_COM"]+"_"+Artif_2014["BANDE_LITT"]
Artif_2014=Artif_2014.dissolve(by="ID_COM_LITT", as_index=False)
Artif_2014["Artif2014_m2"]=Artif_2014["geometry"].area

# Le nom de la commune a été modifié durant les transformations avec Mapshaper. 
# On récupère les noms corrects :
epci=pd.read_csv("BANDE_LITTORALE/BANDE_LITTORALE_SECTEURS_COM_BZH_STATS.csv")
epci=epci[["BANDE_LITT","INSEE_COM", "NOM", 'SIREN_EPCI', 'NOM_EPCI']]
epci["INSEE_COM"]=epci["INSEE_COM"].astype("str")
epci["ID_COM_LITT"]=epci["INSEE_COM"]+"_"+epci["BANDE_LITT"]
epci=epci[['ID_COM_LITT','NOM', 'SIREN_EPCI', 'NOM_EPCI']]
Artif_2014=Artif_2014[['ID_COM_LITT', 'geometry', 'BANDE_LITT', 'INSEE_COM','numdep', 'SURFACE_M2', 'Artif2014_m2']]
Artif_2014=Artif_2014.merge(epci, on="ID_COM_LITT", how="left")

Artif_2014=Artif_2014.to_crs("epsg:4326")
Artif_2014["Part_Artif2014"]=(Artif_2014["Artif2014_m2"]/Artif_2014["SURFACE_M2"])*100
Artif_2014.rename(columns={"NOM":"NOM_COMMUNE"}, inplace=True)
Artif_2014=Artif_2014[['ID_COM_LITT', 'BANDE_LITT', 'numdep', 'INSEE_COM', 'NOM_COMMUNE', 'SIREN_EPCI', 'NOM_EPCI', 'SURFACE_M2','Artif2014_m2', 'Part_Artif2014', 'geometry']]
Artif_2014["SURFACE_M2"]=Artif_2014["SURFACE_M2"].round(0)
Artif_2014["Artif2014_m2"]=Artif_2014["Artif2014_m2"].round(0)
Artif_2014["Part_Artif2014"]=Artif_2014["Part_Artif2014"].round(0)

Artif_2014.to_file("BD_TOPO/TACHE_ARTIF_BZH_2014_BandeLitt_DEF.geojson")
Artif_2014.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1973 entries, 0 to 1972
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   ID_COM_LITT     1973 non-null   object  
 1   BANDE_LITT      1973 non-null   object  
 2   numdep          1973 non-null   object  
 3   INSEE_COM       1973 non-null   object  
 4   NOM_COMMUNE     1973 non-null   object  
 5   SIREN_EPCI      1973 non-null   object  
 6   NOM_EPCI        1963 non-null   object  
 7   SURFACE_M2      1973 non-null   float64 
 8   Artif2014_m2    1973 non-null   float64 
 9   Part_Artif2014  1973 non-null   float64 
 10  geometry        1973 non-null   geometry
dtypes: float64(3), geometry(1), object(7)
memory usage: 169.7+ KB


In [71]:
### Filtrage de la tache artif à moins de 5km du rivage :
Artif_2014_5km=Artif_2014[Artif_2014["BANDE_LITT"]!="5_10km"]
Artif_2014_5km.to_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_BandeLitt_DEF.geojson")

Artif_2014_2024_5km=Artif_2014_2024[Artif_2014_2024["BANDE_LITT"]!="5_10km"]
Artif_2014_2024_5km.to_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_2024_BandeLitt_DEF.geojson")


# =====================================

### Synthèse par secteurs de communes sur la bande littoral de 0 à 5km:

In [72]:
# Fichier CSV des surfaces par communes et par bande littorale :
Artif_2014=gpd.read_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_BandeLitt_DEF.geojson")
Artif_2014_Bilan=Artif_2014[['ID_COM_LITT','BANDE_LITT', 'numdep', 'INSEE_COM', 'NOM_COMMUNE','SIREN_EPCI','NOM_EPCI',
       'SURFACE_M2', 'Artif2014_m2', 'Part_Artif2014']]
Artif_2014_2024=gpd.read_file("BD_TOPO/TACHE_ARTIF_5km_BZH_2014_2024_BandeLitt_DEF.geojson")
Artif_2014_2024_Bilan=Artif_2014_2024[['ID_COM_LITT','Artif2024_m2']]

Artif_Bilan=Artif_2014_Bilan.merge(Artif_2014_2024_Bilan, on="ID_COM_LITT", how="left")
Artif_Bilan["Artif2024_m2"]=Artif_Bilan["Artif2024_m2"].fillna(0)
Artif_Bilan["EvolArtif"]=(Artif_Bilan["Artif2024_m2"]/Artif_Bilan["Artif2014_m2"])*100
Artif_Bilan["EvolArtif"]=Artif_Bilan["EvolArtif"].round(0)
Artif_Bilan.to_csv("Bilan_Artif_BDTopo_COMBandeLitt_2014-2024.csv")
Artif_Bilan.info()
Artif_Bilan.head(30)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1583 entries, 0 to 1582
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID_COM_LITT     1583 non-null   object 
 1   BANDE_LITT      1583 non-null   object 
 2   numdep          1583 non-null   object 
 3   INSEE_COM       1583 non-null   object 
 4   NOM_COMMUNE     1583 non-null   object 
 5   SIREN_EPCI      1583 non-null   object 
 6   NOM_EPCI        1573 non-null   object 
 7   SURFACE_M2      1583 non-null   float64
 8   Artif2014_m2    1583 non-null   float64
 9   Part_Artif2014  1583 non-null   float64
 10  Artif2024_m2    1583 non-null   float64
 11  EvolArtif       1583 non-null   float64
dtypes: float64(5), object(7)
memory usage: 148.5+ KB


,ID_COM_LITT,BANDE_LITT,numdep,INSEE_COM,NOM_COMMUNE,SIREN_EPCI,NOM_EPCI,SURFACE_M2,Artif2014_m2,Part_Artif2014,Artif2024_m2,EvolArtif
0,22002_1_5km,1_5km,22,22002,Andel,200069391,CA Lamballe Terre et Mer,751418.000000000,54802.000000000,7.000000000,0.000000000,0.000000000
1,22012_1_5km,1_5km,22,22012,La Bouillie,200069391,CA Lamballe Terre et Mer,4552934.000000000,298823.000000000,7.000000000,1551.000000000,1.000000000
2,22016_100_200m,100_200m,22,22016,Île-de-Bréhat,NR,None,882124.000000000,434661.000000000,49.000000000,2598.000000000,1.000000000
3,22016_100m,100m,22,22016,Île-de-Bréhat,NR,None,1841480.000000000,497758.000000000,27.000000000,3226.000000000,1.000000000
4,22016_200_500m,200_500m,22,22016,Île-de-Bréhat,NR,None,393226.000000000,209530.000000000,53.000000000,2330.000000000,1.000000000
5,22028_1_5km,1_5km,22,22028,Camlez,200065928,CA Lannion-Trégor Communauté,5038288.000000000,505702.000000000,10.000000000,57843.000000000,11.000000000
6,22042_1_5km,1_5km,22,22042,Coatréven,200065928,CA Lannion-Trégor Communauté,9112.000000000,198.000000000,2.000000000,0.000000000,0.000000000
7,22044_1_5km,1_5km,22,22044,Coëtmieux,200069391,CA Lamballe Terre et Mer,2147798.000000000,330934.000000000,15.000000000,41291.000000000,12.000000000
8,22049_100_200m,100_200m,22,22049,Créhen,200068989,CA Dinan Agglomération,112985.000000000,24431.000000000,22.000000000,0.000000000,0.000000000
9,22049_100m,100m,22,22049,Créhen,200068989,CA Dinan Agglomération,93818.000000000,37156.000000000,40.000000000,0.000000000,0.000000000


### Synthèse par communes :

In [73]:
Artif_BilanCom=Artif_Bilan.groupby(by=["INSEE_COM", "NOM_COMMUNE","SIREN_EPCI", "NOM_EPCI"], as_index=False).agg(
    SURFACE_M2=("SURFACE_M2","sum"),
    Artif2014_m2=("Artif2014_m2","sum"),
    Artif2024_m2=("Artif2024_m2","sum"),
)
Artif_BilanCom["Part_Artif2014"]=(Artif_BilanCom["Artif2014_m2"]/Artif_BilanCom["SURFACE_M2"])*100
Artif_BilanCom["Part_Artif2014"]=Artif_BilanCom["Part_Artif2014"].round(0)
Artif_BilanCom["EvolArtif"]=(Artif_BilanCom["Artif2024_m2"]/Artif_BilanCom["Artif2014_m2"])*100
Artif_BilanCom["EvolArtif"]=Artif_BilanCom["EvolArtif"].round(0)

Artif_BilanCom.to_csv("Bilan_Artif_BDTopo_Com_2014-2024.csv")
Artif_BilanCom.head(10)

,INSEE_COM,NOM_COMMUNE,SIREN_EPCI,NOM_EPCI,SURFACE_M2,Artif2014_m2,Artif2024_m2,Part_Artif2014,EvolArtif
0,22002,Andel,200069391,CA Lamballe Terre et Mer,751418.000000000,54802.000000000,0.000000000,7.000000000,0.000000000
1,22012,La Bouillie,200069391,CA Lamballe Terre et Mer,4552934.000000000,298823.000000000,1551.000000000,7.000000000,1.000000000
2,22028,Camlez,200065928,CA Lannion-Trégor Communauté,5038288.000000000,505702.000000000,57843.000000000,10.000000000,11.000000000
3,22042,Coatréven,200065928,CA Lannion-Trégor Communauté,9112.000000000,198.000000000,0.000000000,2.000000000,0.000000000
4,22044,Coëtmieux,200069391,CA Lamballe Terre et Mer,2147798.000000000,330934.000000000,41291.000000000,15.000000000,12.000000000
5,22049,Créhen,200068989,CA Dinan Agglomération,10576835.000000000,1437404.000000000,73489.000000000,14.000000000,5.000000000
6,22050,Dinan,200068989,CA Dinan Agglomération,669980.000000000,486409.000000000,2247.000000000,73.000000000,0.000000000
7,22054,Erquy,200069391,CA Lamballe Terre et Mer,26409728.000000000,5923883.000000000,240987.000000000,22.000000000,4.000000000
8,22055,Binic-Étables-sur-Mer,200069409,CA Saint-Brieuc Armor Agglomération,15020843.000000000,5905166.000000000,136043.000000000,39.000000000,2.000000000
9,22076,Hénanbihen,200069391,CA Lamballe Terre et Mer,2687334.000000000,158601.000000000,4298.000000000,6.000000000,3.000000000
